# <span style="color:green"> Objective </span>

- To learn to write a work-efficient scan kernel
    - Two-phased balanced tree traversal
    - Aggressive re-use of intermediate results
    - Reducing control divergence with more complex thread index to data index mapping

<hr style="height:2px">

# <span style="color:green"> Improving Efficiency </span>

- Balanced Trees
    - Form a balanced binary tree on the input data and sweep it to and from the root
    - Tree is not an actual data structure, but a concept to determine what each thread does at each step

- For scan:
    - Traverse down from leaves to the root building partial sums at internal nodes in the tree
        - The root holds the sum of all leaves
    - Traverse back up the tree building the output from the partial sums

<hr style="height:2px">

# <span style="color:green"> Parallel Scan - Reduction Phase </span>



__threadIdx.x+1 = 1, 2, 3, 4....__

__stride = 1,__

__index = 1, 3, 5, 7, ...__

<hr style="height:2px">

# <span style="color:green"> Reduction Phase Kernel Code </span>

```cpp

// XY[2*BLOCK_SIZE] is in shared memory
for (unsigned int stride = 1;stride <= BLOCK_SIZE; stride *= 2)
{
    int index = (threadIdx.x+1)*stride*2 - 1;
    if(index < 2*BLOCK_SIZE)
    XY[index] += XY[index-stride];
    __syncthreads();
}

```

![alt tag](img/3.png)
<hr style="height:2px">

# <span style="color:green"> Parallel Scan - Post Reduction Reverse Phase </span>

![alt tag](img/6.png)

![alt tag](img/7.png)

<hr style="height:2px">

# <span style="color:green"> Putting it Together </span>

![alt tag](img/8.png)
<hr style="height:2px">

# <span style="color:green"> Post Reduction Reverse Phase Kernel Code </span>

```cpp

for (unsigned int stride = BLOCK_SIZE/2; stride > 0; stride /= 2) {
    __syncthreads();
    int index = (threadIdx.x+1)*stride*2 - 1;
    if(index+stride < 2*BLOCK_SIZE) {
        XY[index + stride] += XY[index];
    }
}
__syncthreads();
if (i < InputSize) Y[i] = XY[threadIdx.x];

```

__First iteration for 16-element section__

__threadIdx.x = 0__

__stride = BLOCK_SIZE/2 = 8/2 = 4__

__index = 8-1 = 7__

<hr style="height:2px">

<footer>
<cite> GPU NVIDIA Teaching Kit - University of Illinois </cite>
</footer>